<a href="https://colab.research.google.com/github/JosuZx13/JosuZx13/blob/main/MachineLearningAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TFG - ALBA CASILLAS**

**Machine Learning API**

In [ ]:
from google.colab import drive
drive.mount("/content/drive") # Se monta el Drive

from google.colab import files

In [ ]:
!cp /content/drive/MyDrive/TFG_Alba/rnapi.py /content

In [ ]:
!pip install colored
# Dependencias
!pip install -U kaleido

# ===========================================
# SEMILLA AJUSTADA A 1
# ===========================================

In [ ]:
import random
random.seed(1)

# ========================================
# ~ READ DATA
# ========================================


In [ ]:
data_object = rna.create_tcga_query(project = "TCGA-BRCA", data_category = "TRANSCRIPTOME PROFILING", data_type = "Gene Expression Quantification",
                        experimental_strategy = "RNA-Seq")

# ProcessTCGA.download_data(data_object)

# data = ProcessTCGA.read_rna("TCGA-BRCA")

# data.to_csv(r'D:/TFG_archivos/info_y_copias/OOB_API/brca_counts.txt', sep='\t')

count_data = rna.read_file("brca_counts.txt", index = "gene_id", sep="\t")

clinical_data = rna.tcga_read_clinical("TCGA-BRCA/nationwidechildrens.org_clinical_patient_brca.txt")

gene_info = rna.read_file("gene_info_brca.tsv", index = "gene_id", sep="\t")

# ===========================================
# ~ CREATE DATA OBJECT
# ===========================================

In [ ]:
data_object = rna.create_data_object(count_data, obs_ = clinical_data, var_ = gene_info)

data_object.summary_object()
print("\n\n")


# Define a list of Log to create a Pipeline
logs = []

# ============================================================
# ~ CLEANING CLINICAL DATA
# ============================================================

In [ ]:
data_object = rna.obs_replace(data_object, to_replace = [ "[Discrepancy]", "[Not Available]", "[Not Applicable]"])

data_object.summary_object()

# ============================================================
# ~ FILTER SAMPLES
# ============================================================

In [ ]:
# Check distribution of gender and histological_type

print("Distribution of gender:\n")

rna.count_types(data_object.get_obs(), "gender")


print("\n\n")
print("Distribution of histological type:\n")


rna.count_types(data_object.get_obs(), "histological_type")


print("\n\n")


# Filter samples to female with the most frequent histological subtype and sample subtype

series_condition_rows = (data_object.get_obs()["histological_type"].str.lower() == 'infiltrating ductal carcinoma') & (data_object.get_obs()["gender"].str.lower() == 'female')

extracted_by_series = rna.data_selection_series(series_condition_rows, data_object.get_obs())

data_object = rna.set_obs(data_object, extracted_by_series)

# data_object.summary_object()

rna.count_types(data_object.get_obs(), "pathologic_stage")

# ============================================================
# ~ ASSIGN A LABEL TO EACH EXAMPLE OF THE DATASET
# ============================================================

In [ ]:
obs_with_new_column = rna.add_column(data_object.get_obs(), "label", rna.ut.np.ones(data_object.obs_num_rows(), dtype=int))


mask_pathologic = data_object.get_obs()["pathologic_stage"].str.lower().isin(['stage i','stage ia','stage ib','stage ii','stage iia','stage iib'])

obs_with_new_column.loc[mask_pathologic, "label"] = 0


data_object = rna.set_obs(data_object, obs_with_new_column)



rna.count_types(data_object.get_obs(), "label")

# ============================================================
# ~ CLINICAL FEATURES
# ============================================================

In [ ]:
clinical = rna.data_projection_list(data_object.get_obs(), ["age_at_initial_pathologic_diagnosis","race"])


for i in clinical.columns.drop("age_at_initial_pathologic_diagnosis"):
    print(data_object.get_obs().groupby([i, "label"]).count().iloc[:,1])
    

data_object.get_obs()["age_at_initial_pathologic_diagnosis"] = data_object.get_obs()["age_at_initial_pathologic_diagnosis"].astype(float)


data_subtype = rna.copy_object(data_object.get_obs())


# 0 means early stage, 1 means late
fig = rna.box_plot(data_object.get_obs(), x = "label", y= "age_at_initial_pathologic_diagnosis")

#rna.show_figure(fig, xlabel = "label", ylabel = "stage", legend = False, title = "age at diagnosis vs. stage")

rna.save_image(fig, fig_name = "ml_box_plot", xlabel = "label", ylabel = "stage", legend = False, title = "age at diagnosis vs. stage")

# ============================================================
# ~ DROP GENE COLUMNS CONTAINING MORE THAN 10% NA
# ============================================================

In [ ]:
data_object = rna.counts_replace(data_object, 0)


# Calculate the thresh
perc = 10.0
min_count = int(((100-perc)/100)*data_object.counts_num_columns() + 1)

# mod_data = data.dropna( axis = 0, thresh = min_count)

data_object = rna.counts_drop_nan_by_thresh(data_object, 0, min_count)


data_object = rna.counts_replace_nan(data_object)



print("IS THERE NAN VALUES???? ")

print(data_object.get_counts().isnull().values.any())

print(data_object.get_counts().isnull().sum().sum())

# ============================================================
# GENERATE YOU PROCESSED FEATURE VECTOR FOR EACH EXAMPLE
# ============================================================

In [ ]:
data_transpose = rna.transpose(data_object.get_counts())

# Calculate variance of genes
#var = dict(zip(data_transpose.var().index, data_transpose.var()))

# index without values and list are index_base, and var() is Series
var = rna.list_zip(list(data_transpose.var().index.values), data_transpose.var().tolist())

# Sort genes by variance

var_sort = sorted(var.items(), key=lambda d: d[1], reverse=True)

# Find top 1000 genes

var_1000 = [i for i, j in var_sort[0:1000]]


# FILTERS GNAMES TO GENES_100 (1000)

gnames = rna.data_zip(data_object.get_var(), "gene_id", "gene_name")


genes_1000 = rna.filter_dictionary(gnames, var_1000)


genes_7 = ["MYC", "PIK3CA", "TP53", "BRCA1", "BRCA2", "CDH1", "PTEN"]
lkgenes_7 = []

for gen in genes_7:
    
    if gen not in list(genes_1000.values()):
        
        kgenes_7 = list(gnames.keys())[list(gnames.values()).index(gen)]
        lkgenes_7.append(kgenes_7)


# Select 1007 genes
print(len(list(genes_1000.keys())))

data_object = rna.counts_selection(data_object, (list(genes_1000.keys()) + lkgenes_7))


data_object.summary_object()

# ============================================================
# REMOVE HIGH CORRELATED VARIABLES
# ============================================================

In [ ]:
print("Find high Correlation\n")
# Find high correlation

corr = rna.var_correlation(data_object.get_counts())


fig = rna.heatmap(x = corr.columns, y = corr.index, z = rna.ut.np.array(corr))

# rna.show_figure(fig, xlabel = "x axis", ylabel = "y axis", legend = False, title = "Correlation between variables")

rna.save_image(fig, fig_name = "ml_heatmap", xlabel = "x axis", ylabel = "y axis", legend = False, title = "Correlation between variables")

genes = rna.remove_correlation(data_object.get_counts(), 0.6)

# genes = qa.transpose(genes)

# print(genes.shape) (878, 506)

#genes = genes[genes.index.isin(data_subtype.index)]

genes = rna.data_selection_list(genes, list(data_subtype.index.values))

final_data = genes

# def update_column_values
final_data["label"] = data_subtype.label

print(final_data)

"""# =============================================================================
#                              MACHINE LEARNING
# =============================================================================
"""

# ============================================================
# MODELO UNO
# ============================================================

In [ ]:
print("MODELO UNO:\n")

data = rna.ModelSelection(final_data.copy())

data.split_data()

data.get_train_test_sample(test_size = 0.2, resample = True)

data.standarize()

print("\n\n")
models = []

models.append(rna.model_.LogisticRegression('LR', solver='lbfgs', max_iter=1000))
models.append(rna.model_.RandomForest('RF', n_estimators=500))
models.append(rna.model_.ScikitLearnModel('GNB', rna.model_.GaussianNB()))
models.append(rna.model_.SupportVectorClassif('SVC', kernel = 'sigmoid',probability=True))
models.append(rna.model_.MLPerceptron('MLP', max_iter = 500, hidden_layer_sizes=(5,5,5)))

model_results = rna.ut.pd.DataFrame()


for model_for in models:   
    
    print('Fitting: ', model_for.get_name())
    
    model_for.fit(data.get_X_train(), data.get_Y_train())
    
    # Devuelve la probabilidad de que sea 0 o 1
    predictions_proba = data.predict_proba(model_for)
    # Nos quedamos con los que superen el 50% de la probabilidad
    predictions = data.get_predictions(predictions_proba, 0.5)    
    
    metrics = model_for.get_metrics(data.get_Y_test(), predictions, predictions_proba)
   
    model_results = rna.ut.pd.concat([model_results, metrics], axis=0) 


print(model_results)

# ============================================================
# MODELO DOS
# ============================================================

In [ ]:
print("MODELO DOS:\n")

tmp = final_data.copy()

data7 = tmp.drop(lkgenes_7,axis=1)

data = rna.ModelSelection(data7)

data.split_data()

data.get_train_test_sample(test_size = 0.2, resample = True)

data.standarize()

print("\n\n")
models = []

models.append(rna.model_.LogisticRegression('LR', solver='lbfgs', max_iter=1000))
models.append(rna.model_.RandomForest('RF', n_estimators=500))
models.append(rna.model_.ScikitLearnModel('GNB', rna.model_.GaussianNB()))
models.append(rna.model_.SupportVectorClassif('SVC', kernel = 'sigmoid',probability=True))
models.append(rna.model_.MLPerceptron('MLP', max_iter = 500, hidden_layer_sizes=(5,5,5)))

model_results = rna.ut.pd.DataFrame()



for model_for in models:   
    
    print('Fitting: ', model_for.get_name())
    
    model_for.fit(data.get_X_train(), data.get_Y_train())
    
    # Devuelve la probabilidad de que sea 0 o 1
    predictions_proba = data.predict_proba(model_for)
    # Nos quedamos con los que superen el 50% de la probabilidad
    predictions = data.get_predictions(predictions_proba, 0.5)    
    
    metrics = model_for.get_metrics(data.get_Y_test(), predictions, predictions_proba)
   
    model_results = rna.ut.pd.concat([model_results, metrics], axis=0) 

print(model_results)

# ============================================================
# MODELO TRES
# ============================================================

In [ ]:
print("MODELO TRES:\n")

tmp = final_data.copy()

label = tmp.label

tmp = rna.ut.pd.concat([tmp.iloc[:,:100],tmp.loc[:,lkgenes_7]], axis=1)
dup_cols = tmp.columns[tmp.columns.duplicated()]
tmp.drop(columns = dup_cols, inplace = True)

tmp['label'] = label

data107 = tmp.copy()

data = rna.ModelSelection(data107)

data.split_data()

data.get_train_test_sample(test_size = 0.2, resample = True)

data.standarize()

print("\n\n")
models = []

models.append(rna.model_.LogisticRegression('LR', solver='lbfgs', max_iter=1000))
models.append(rna.model_.RandomForest('RF', n_estimators=500))
models.append(rna.model_.ScikitLearnModel('GNB', rna.model_.GaussianNB()))
models.append(rna.model_.SupportVectorClassif('SVC', kernel = 'sigmoid',probability=True))
models.append(rna.model_.MLPerceptron('MLP', max_iter = 500, hidden_layer_sizes=(5,5,5)))

model_results = rna.ut.pd.DataFrame()



for model_for in models:   
    
    print('Fitting: ', model_for.get_name())
    
    model_for.fit(data.get_X_train(), data.get_Y_train())
    
    # Devuelve la probabilidad de que sea 0 o 1
    predictions_proba = data.predict_proba(model_for)
    # Nos quedamos con los que superen el 50% de la probabilidad
    predictions = data.get_predictions(predictions_proba, 0.5)    
    
    metrics = model_for.get_metrics(data.get_Y_test(), predictions, predictions_proba)
   
    model_results = rna.ut.pd.concat([model_results, metrics], axis=0) 

print(model_results)

# ============================================================
# MODELO CUATRO
# ============================================================

In [ ]:
print("MODELO CUATRO: ")


tmp = final_data.copy()

label = tmp.label

data = rna.ut.pd.concat([tmp.iloc[:,:50],tmp.loc[:,lkgenes_7]], axis=1)
dup_cols = tmp.columns[tmp.columns.duplicated()]
tmp.drop(columns = dup_cols, inplace = True)

tmp['label']=label

data57 = tmp.copy()   

data = rna.ModelSelection(data57)

data.split_data()

data.get_train_test_sample(test_size = 0.2, resample = True)

data.standarize()

print("\n\n")
models = []

models.append(rna.model_.LogisticRegression('LR', solver='lbfgs', max_iter=1000))
models.append(rna.model_.RandomForest('RF', n_estimators=500))
models.append(rna.model_.ScikitLearnModel('GNB', rna.model_.GaussianNB()))
models.append(rna.model_.SupportVectorClassif('SVC', kernel = 'sigmoid',probability=True))
models.append(rna.model_.MLPerceptron('MLP', max_iter = 500, hidden_layer_sizes=(5,5,5)))

model_results = rna.ut.pd.DataFrame()


for model_for in models:   
    
    print('Fitting: ', model_for.get_name())
    
    model_for.fit(data.get_X_train(), data.get_Y_train())
    
    # Devuelve la probabilidad de que sea 0 o 1
    predictions_proba = data.predict_proba(model_for)
    # Nos quedamos con los que superen el 50% de la probabilidad
    predictions = data.get_predictions(predictions_proba, 0.5)    
    
    metrics = model_for.get_metrics(data.get_Y_test(), predictions, predictions_proba)
   
    model_results = rna.ut.pd.concat([model_results, metrics], axis=0) 

print(model_results)

# ============================================================
# OPTIMIZANDO RANDOM FOREST AJUSTANDO SUS HIPERPARAMETROS
# ============================================================

In [ ]:
data = rna.ModelSelection(data107)

data.split_data()

data.get_train_test_sample(test_size = 0.2, resample = True)

data.standarize()

random_forest = rna.model_.RandomForest('RF')

random_forest.hyperparameter_tuning(data.get_X_train(), data.get_Y_train(), n_estimators = [10,50,100,500])

random_forest.fit(data.get_X_train(), data.get_Y_train())

predictions_proba = data.predict_proba(random_forest)
predictions = data.get_predictions(predictions_proba, threshold = 0.5)

metrics = random_forest.get_metrics(data.get_Y_test(), predictions, predictions_proba)

print(metrics)

# ========================================================
# CURVA PRECISION-RECALL
# ========================================================

In [ ]:
fig = rna.plot_prec_recall_vs_thresh(data.get_Y_test() , predictions_proba)

# rna.show_figure(fig, xlabel = "Threshold", title = 'Precision-Recall vs Tresholds Curve')

rna.save_image(fig, fig_name="prec_recall_vs_thresh", xlabel = "Threshold", title = 'Precision-Recall vs Tresholds Curve')

# ============================================================
# MATRIZ DE CONFUSION
# ============================================================

In [ ]:
cm = data.confusion_matrix(predictions)

fig = rna.plot_confusion_matrix(cm)

#rna.show_figure(fig, title = "Confussion Matrix")

rna.save_image(fig, fig_name ="confussion_matrix", title="Confussion Matrix")

# ============================================================
# CURVA ROC
# ============================================================

In [ ]:
roc_auc = data.calc_auc(predictions)
title = "ROC CURVE " + f'(AUC={roc_auc:.4f})'

fig = rna.plot_roc(data.get_Y_test(),predictions_proba)
# rna.show_figure(fig, xlabel = "False Positive Rate", ylabel = "True Positive Rate",
#                          x_ini_range = 0, x_fin_range = 1,
#                          y_ini_range = 0, y_fin_range = 1, title = title)

rna.save_image(fig, fig_name="roc", xlabel = "False Positive Rate", ylabel = "True Positive Rate",
                          x_ini_range = 0, x_fin_range = 1,
                          y_ini_range = 0, y_fin_range = 1, title = title)

# ============================================================
# CURVA PRECISION-RECALL
# ============================================================

In [ ]:
pr_auc = data.calc_auc(predictions, curve = "precision-recall")
title = "PR CURVE " + f'(AUC={pr_auc:.4f})'

fig = rna.plot_prc(data.get_Y_test(),predictions_proba)
#rna.show_figure(fig, xlabel = "Recall", ylabel = "Precision", x_ini_range = 0, x_fin_range = 1,
#                    y_ini_range = 0, y_fin_range = 1, title = 'PR Curve')

rna.save_image(fig, fig_name="prc", xlabel = "Recall", ylabel = "Precision", x_ini_range = 0, x_fin_range = 1,
                    y_ini_range = 0, y_fin_range = 1, title = 'PR Curve')